In [24]:
pip install geopy

    100% |████████████████████████████████| 122kB 5.5MB/s ta 0:00:01
    100% |████████████████████████████████| 40kB 5.9MB/s eta 0:00:01
You are using pip version 18.1, however version 23.1.2 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [2]:
#Import all the necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from statsmodels.formula.api import ols
import statsmodels.api as sm
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler, StandardScaler
from sklearn.linear_model import LinearRegression, ElasticNet, Ridge, Lasso
from sklearn.impute import SimpleImputer
from geopy.geocoders import Nominatim


The first step we carried out was to import the data and check if the data is clean. 
1. Import the data and view the imported data
2. The data is clean and no columns are missing any values. 
3. All column entries except date column is the right datatype which we want it to be. 
4. We converted the date column to datetime datatype.

In [3]:
#import the house data and view the imported data
house_data = pd.read_csv('https://raw.githubusercontent.com/Pelumioluwa/MBAN6110Class/main/Class_code/kc_house_data.csv')
house_data.head()

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,7129300520,20141013T000000,221900.0,3,1.00,1180,5650,1.0,0,0,...,7,1180,0,1955,0,98178,47.5112,-122.257,1340,5650
1,6414100192,20141209T000000,538000.0,3,2.25,2570,7242,2.0,0,0,...,7,2170,400,1951,1991,98125,47.7210,-122.319,1690,7639
2,5631500400,20150225T000000,180000.0,2,1.00,770,10000,1.0,0,0,...,6,770,0,1933,0,98028,47.7379,-122.233,2720,8062
3,2487200875,20141209T000000,604000.0,4,3.00,1960,5000,1.0,0,0,...,7,1050,910,1965,0,98136,47.5208,-122.393,1360,5000
4,1954400510,20150218T000000,510000.0,3,2.00,1680,8080,1.0,0,0,...,8,1680,0,1987,0,98074,47.6168,-122.045,1800,7503


In [4]:
#check the information of the data see if it is clean
house_data.info() 


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21613 entries, 0 to 21612
Data columns (total 21 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   id             21613 non-null  int64  
 1   date           21613 non-null  object 
 2   price          21613 non-null  float64
 3   bedrooms       21613 non-null  int64  
 4   bathrooms      21613 non-null  float64
 5   sqft_living    21613 non-null  int64  
 6   sqft_lot       21613 non-null  int64  
 7   floors         21613 non-null  float64
 8   waterfront     21613 non-null  int64  
 9   view           21613 non-null  int64  
 10  condition      21613 non-null  int64  
 11  grade          21613 non-null  int64  
 12  sqft_above     21613 non-null  int64  
 13  sqft_basement  21613 non-null  int64  
 14  yr_built       21613 non-null  int64  
 15  yr_renovated   21613 non-null  int64  
 16  zipcode        21613 non-null  int64  
 17  lat            21613 non-null  float64
 18  long  

In [5]:
#convert the date cloumn to datetime type 
house_data['date'] = pd.to_datetime(house_data['date'])

In [6]:
#check the summary of the columns
house_data['zipcode'].describe(include='all',datetime_is_numeric=True)

count    21613.000000
mean     98077.939805
std         53.505026
min      98001.000000
25%      98033.000000
50%      98065.000000
75%      98118.000000
max      98199.000000
Name: zipcode, dtype: float64

The second step we carried out was feature engineering 
1. Utilize the longtitude and latitude to determine the address of the house 
2. Use year of renovation to determine how recently it was renovated 
3. Use the size of the lot and living space to determine how big the house is 
. If the living space size is relatively large compared to the lot size, it may indicate that the house has more interior space in relation to the land it occupies. Conversely, if the lot size is significantly larger than the living space size, it may suggest a smaller house or a larger outdoor area.



using both zip code and address is
zipcode The ZIP code can be a useful starting point for understanding the general area and market conditions. It can provide an overview of the neighborhood or region where the house is located. ZIP codes are often used in real estate analysis to compare property prices, demographics, and market trends within a particular area. However, using ZIP code alone may not capture the specific characteristics or desirability of a particular address within that ZIP code
using address specific address of the house offers a higher level of accuracy and granularity. It takes into account the unique attributes of the property, such as its proximity to amenities, schools, transportation, and other local factors that can influence the price. The address provides a more precise location for conducting comparable market analyses and evaluating the specific features and qualities of the property.

What features we want to use 
determine how big the house is using the size of the lot and living space
number of bathrooms 
number of bedrooms
has a waterfront or not 
how good the view is 
number of floors
condition of the house
the grade of the house
year renovated
zip

In [38]:
#Get the exact address of the houses with the longtitude and latitude data
geolocator = Nominatim(user_agent="my_app")
for index, row in house_data.iterrows():
    latitude = row['lat']
    longitude = row['long']
    # Perform reverse geocoding for each coordinate pair
    location = geolocator.reverse((latitude, longitude))
    address = location.address
    # Add the address to a new column in the DataFrame
    house_data.at[index, 'address'] = address

house_data.head()


In [7]:
house_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21613 entries, 0 to 21612
Data columns (total 21 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   id             21613 non-null  int64         
 1   date           21613 non-null  datetime64[ns]
 2   price          21613 non-null  float64       
 3   bedrooms       21613 non-null  int64         
 4   bathrooms      21613 non-null  float64       
 5   sqft_living    21613 non-null  int64         
 6   sqft_lot       21613 non-null  int64         
 7   floors         21613 non-null  float64       
 8   waterfront     21613 non-null  int64         
 9   view           21613 non-null  int64         
 10  condition      21613 non-null  int64         
 11  grade          21613 non-null  int64         
 12  sqft_above     21613 non-null  int64         
 13  sqft_basement  21613 non-null  int64         
 14  yr_built       21613 non-null  int64         
 15  yr_renovated   2161